In [ ]:
# Now create Workspace
# Create Compute Target
# Create Experiment
# Create environment to execute your code
# Register model in workspace
# Deploying the model
# Prepare your test data
# Real time scoring using RestAPI Call

In [ ]:
from azureml.core import Workspace, ComputeTarget, Datastore, Dataset, Experiment
from azureml.core.compute import AmlCompute
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

In [ ]:
# #details of your AML workspace
subscription_id = "0f644ff4-434a-4446-a414-16ddb2b713ee"
resource_group = "rg-sbihackathon"
workspace = "ml-demo-explore"

aml_compute_target = "azureml-train-cluster-01" #ALL SMALL LETTER, NO underscore, 16ws long only.
experiment_name= 'demo_expirement'

In [ ]:

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)
try:
    ws = Workspace.from_config() #fetches details from 'config.json'
    # #alternate way to configure workspace
    # ws = Workspace.get(name='0f644ff4-434a-4446-a414-16ddb2b713ee',
    #                     subscription_id='rg-sbihackathon',
    #                     resource_group='ml-demo-explore')
    
    print('Workspace already exist!')
    # print('Workspace already exist!',ws, sep='\n')
    print('\nCompute Targets:')
    if ws.compute_targets != {} or len(ws.compute_targets) != 0:
        for compute_name in ws.compute_targets:
            compute = ws.compute_targets[compute_name]
            # compute = ComputeTarget._get(ws, compute_name)
            print("\t",compute.name, ":", compute.type)
    else:
        print("\tNone")

    print('Datastores:')
    if ws.datastores != {} or len(ws.datastores) != 0:
        for datastore_name in ws.datastores:
            # datastore = ws.datastores[datastore_name]
            datastore = Datastore.get(ws, datastore_name)
            print("\t",datastore.name, ':', datastore.datastore_type)
    else:
        print("\tNone")
        
    print('Datasets:')
    if len(list(ws.datasets.keys())) != 0:
        for dataset_name in list(ws.datasets.keys()):
            dataset = Dataset.get_by_name(ws, dataset_name)
            print('\t',dataset.name)
    else:
        print("\tNone")
    
    print('Experiments:')
    # print(ws.experiments)
    if ws.experiments != {} or len(ws.experiments) != 0:
        for experiment_name in ws.experiments:
            # experiment = Experiment._get_base_info_dict() (ws, experiment_name)
            experiment = ws.experiments[experiment_name]
            print("\t",experiment.name, ':', experiment.workspace.name)
    else:
        print("\tNone")
        
except Exception as e:
    print("Exception", e)
    # ws=Workspace.create(workspace_name, 
    #                 resource_group=resource_name,
    #                 create_resource_group=True,
    #                 subscription_id=subscriptionID,
    #                 location="East US")
    # ws.write_config('.azureml')
    
print("\n",ws, sep='\n')

In [ ]:
# Create Compute Target
from azureml.core.compute_target import ComputeTargetException

try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
    
except ComputeTargetException as ComputeNotFound:
    print(ComputeNotFound.message)
    
    print("Creating new ComputeTarget :",aml_compute_target)
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                            min_nodes = 1, 
                                                            max_nodes = 4,
                                                            idle_seconds_before_scaledown=3000)    
    # aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
except Exception as e:
    print(e)
    
print("Done")